In [43]:
import os
import pandas as pd
import ipywidgets as widgets

In [44]:
df = pd.read_csv('ATLAS-Grid view.csv')

df['Foodservice menu prominence -2019'] = pd.to_numeric(df['Foodservice menu prominence -2019'].str.replace('%', '')) / 100

In [45]:
metrics = ['Overall Sustainability Score', 'Edible Portion', 'Mercury Content', 'Foodservice menu prominence -2019']
output = widgets.Output()

sust_slider = widgets.FloatSlider(value = 0.25, min = 0, max = 1, step = 0.01, description = 'Sustainability')
welf_slider = widgets.FloatSlider(value = 0.25, min = 0, max = 1, step = 0.01, description = 'Animal Welfare')
health_slider = widgets.FloatSlider(value = 0.25, min = 0, max = 1, step = 0.01, description = 'Public Health')
market_slider = widgets.FloatSlider(value = 0.25, min = 0, max = 1, step = 0.01, description = 'Market Size')

s_widgets = [sust_slider, welf_slider, health_slider, market_slider]

dropdown = widgets.Dropdown(options = [('Treat missing values as zero', False), ('Ignore missing values', True)])
ignore_missing = False

def compute_scores():
    ignore_missing = dropdown.value
    raw_weights = [s_widget.value for s_widget in s_widgets]
    sum_of_weights = sum(raw_weights)
    weights = [raw_weight / sum_of_weights for raw_weight in raw_weights]
    
    # This is completely the wrong formula - this version is just a proof of concept
    if ignore_missing:
        df['Score'] = pd.DataFrame([pd.to_numeric(df[metric]) * weight for weight, metric in zip(weights, metrics)]).mean(axis = 0)        
    else:
        df['Score'] = pd.DataFrame([pd.to_numeric(df[metric]) * weight for weight, metric in zip(weights, metrics)]).sum(axis = 0)
    df.sort_values('Score', inplace = True, ascending = False)
    
    with output:
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(df[['Archetype', 'Score', 'Overall Sustainability Score', 'Edible Portion', 'Mercury Content', 'Foodservice menu prominence -2019']])

def handler(change):
    output.clear_output()
    compute_scores()
    
for s_widget in s_widgets:
    s_widget.observe(handler, names = 'value')
dropdown.observe(handler, names = 'value')

In [46]:
for s_widget in s_widgets:
    display(s_widget)

display(dropdown)
display(output)
compute_scores()

FloatSlider(value=0.25, description='Sustainability', max=1.0, step=0.01)

FloatSlider(value=0.25, description='Animal Welfare', max=1.0, step=0.01)

FloatSlider(value=0.25, description='Public Health', max=1.0, step=0.01)

FloatSlider(value=0.25, description='Market Size', max=1.0, step=0.01)

Dropdown(options=(('Treat missing values as zero', False), ('Ignore missing values', True)), value=False)

Output()